In [1]:
import pandas as pd
from pathlib import Path
import numpy as np

In [5]:
from sklearn.metrics import cohen_kappa_score
completefast = []
completepub = []
def get_concordance(scheme):
    pubmlst = pd.read_csv(f'pubmlst_output/{scheme}.csv', index_col=0)
    pubmlst.sort_index(inplace=True)
    pubmlst.drop(columns='Scheme', inplace=True)
    ## Especial case in
    if scheme == 'ypseudotuberculosis':
        pubmlst.columns = pubmlst.columns.str.replace(r'^Yersinia_pseudotuberculosis_MLST_', '', regex=True)
    if scheme == 'mcatarrhalis':
        pubmlst.columns = pubmlst.columns.str.replace(r'^Moraxella_catarrhalis_MLST_', '', regex=True)
#         print(pubmlst)
    ##
    comp_pubmlst = pubmlst.copy()
    comp_pubmlst[comp_pubmlst.columns] = comp_pubmlst[comp_pubmlst.columns].apply(pd.to_numeric, errors='coerce')
    typed_pubmlst = comp_pubmlst.loc[comp_pubmlst['ST'].notnull()]
    fastmlst = pd.read_csv(f'fastmlst_output/{scheme}.csv', index_col=0)
    fastmlst.sort_index(inplace=True)
    fastmlst.drop(columns='Scheme', inplace=True)
    comp_fastmlst = fastmlst.copy()
    comp_fastmlst[comp_fastmlst.columns] = comp_fastmlst[comp_fastmlst.columns].apply(pd.to_numeric, errors='coerce')
    # FastMLST extra column drop
    extracolumn = set(comp_fastmlst.columns) - set(comp_pubmlst.columns)
    if extracolumn:
        comp_fastmlst.drop(columns=extracolumn, inplace=True)
    typed_fastmlst = comp_fastmlst.loc[comp_fastmlst['ST'].notnull()]

    # Check if any program make more typing
    extrain = None
    tokapa_pubmlst = typed_pubmlst.copy()
    if len(typed_pubmlst) != len(typed_fastmlst):
        if len(typed_pubmlst) > len(typed_fastmlst):
            extrain = 'pubmlst'
            extragenometyped = list(set(typed_pubmlst.index) - set(typed_fastmlst.index))
            print(extragenometyped)
            print(fastmlst.loc[extragenometyped].to_csv(f'{scheme}_nottypedby_fastmlst.csv'))
            print('-'*20)
            typed_pubmlst = typed_pubmlst.drop(extragenometyped)
        else:
            extrain = 'fastmlst'
            extragenometyped = list(set(typed_fastmlst.index) - set(typed_pubmlst.index))
            comp_fastmlst = comp_fastmlst.drop(extragenometyped)
#         print(scheme ,extrain, extragenometyped)
#     print(typed_pubmlst.columns)
#     print(typed_fastmlst.columns)
    # Check concordance
    try:
        concordance = typed_fastmlst.sort_index(axis=1) == typed_pubmlst.sort_index(axis=1)
    except Exception as e:
#         print(typed_fastmlst.sort_index(axis=1))
        print(scheme,typed_pubmlst.sort_index(axis=1))
    totalhits = len(typed_pubmlst['ST'])
    positivehit = int(concordance['ST'].loc[concordance['ST'] == True].count())
    negativehit = int(concordance['ST'].loc[concordance['ST'] == False].count())
    if totalhits == 0:
#         print(f'{scheme} no assigned pubmlst')
        summary = {'scheme': scheme, 
                'concordance': 'pubmlst_NA',
                'pubmlst_typed': totalhits,
                'pubmlst_typed': positivehit,
                'kappa': 'NA',
               }
        return summary
    if extrain == 'pubmlst':
        totalhits += len(extragenometyped)
    elif extrain == 'fastmlst':
        positivehit += len(extragenometyped)
    raw_fastmlst_to_compare = fastmlst.loc[tokapa_pubmlst.index]
#     print(tokapa_pubmlst.sort_index(axis=1)['ST'].to_string())
#     if scheme == 'sagalactiae':
#     print(tokapa_pubmlst.sort_index(axis=1)['ST'].astype(str))
#     print(raw_fastmlst_to_compare.sort_index(axis=1)['ST'].astype(str))
    
    concordancefactor = (positivehit-negativehit)/totalhits
    summary = {'scheme': scheme, 
            'concordance': concordancefactor,
            'pubmlst_typed': totalhits,
            'fastmlst_typed': positivehit,
            'kappa': cohen_kappa_score(list(tokapa_pubmlst.sort_index(axis=1)['ST'].astype(int, errors='ignore').astype(str)), list(raw_fastmlst_to_compare.sort_index(axis=1)['ST'].astype(int,errors='ignore').astype(str)))
           }
    completefast.extend(list(raw_fastmlst_to_compare.sort_index(axis=1)['ST'].astype(int,errors='ignore').astype(str)))
    completepub.extend(list(tokapa_pubmlst.sort_index(axis=1)['ST'].astype(int, errors='ignore').astype(str)))
    return summary
#     print(f'Concordance among typed assemblies in {scheme}: {concordancefactor} ({positivehit}/{totalhits})')

In [6]:
pubmlstpath = Path('pubmlst_output/')
data_sum = []
for sch in pubmlstpath.glob('*.csv'):
    data_sum.append(get_concordance(sch.stem))
concord = pd.DataFrame(data_sum)
concord.set_index('scheme', inplace=True)
# concord
concord.to_csv('concordance.csv')

['GCF_014397205.fna.gz']
None
--------------------
['GCF_900076365.fna.gz', 'GCF_001056585.fna.gz', 'GCF_001055075.fna.gz']
None
--------------------
['GCF_003213725.fna.gz']
None
--------------------
['GCF_001280245.fna.gz']
None
--------------------
['GCF_002529345.fna.gz']
None
--------------------
['GCF_001694045.fna.gz', 'GCF_009934315.fna.gz', 'GCF_009933805.fna.gz', 'GCF_014875305.fna.gz', 'GCF_000290335.fna.gz', 'GCF_900475355.fna.gz', 'GCF_001546715.fna.gz', 'GCF_001694815.fna.gz', 'GCF_003288055.fna.gz', 'GCF_001190845.fna.gz', 'GCF_000289195.fna.gz', 'GCF_002937095.fna.gz', 'GCF_011326915.fna.gz', 'GCF_000347415.fna.gz', 'GCF_001348575.fna.gz', 'GCF_000299135.fna.gz', 'GCF_001693965.fna.gz', 'GCF_001856445.fna.gz', 'GCF_000289355.fna.gz', 'GCF_001694125.fna.gz', 'GCF_000288295.fna.gz', 'GCF_008710735.fna.gz', 'GCF_001217725.fna.gz', 'GCF_003287995.fna.gz', 'GCF_001694435.fna.gz', 'GCF_002812465.fna.gz', 'GCF_001856455.fna.gz', 'GCF_001694355.fna.gz', 'GCF_001694495.fna.gz', 

In [30]:
# general concordance 
# pubmlst = 27_359
# fastmlst = 27_101
# concordance_total = fastmlst/pubmlst
# rfastmlst = list(range(fastmlst)) + [1]*(pubmlst-fastmlst)
# rpubmlst = list(range(fastmlst)) + [0]*(pubmlst-fastmlst)
# print(len(rpubmlst), len(rfastmlst))
print('Overall kappa:', cohen_kappa_score(completefast, completepub))

Overall kappa: 0.9904763615702162


In [44]:
#Novel or new ST
def new_ST(scheme):
    fastmlst = pd.read_csv(f'fastmlst_output/{scheme}.csv', index_col=0)
    fastmlst.sort_index(inplace=True)
    return len(fastmlst.loc[fastmlst['ST'].astype(str).str.startswith('new_ST')])
def new_alleles(scheme):
    fastmlst = pd.read_csv(f'fastmlst_output/{scheme}.csv', index_col=0)
    fastmlst.sort_index(inplace=True)
    return len(fastmlst.loc[fastmlst['ST'].astype(str).str.startswith('new_alleles')])
# print(novel('efaecium')['ST'].value_counts())
extra_fastmlst = []
for sch in pubmlstpath.glob('*.csv'):
#     data_novelST.append(new_ST(sch.stem))
#     data_novelallele.append(new_alleles(sch.stem))
    datadict = {'Scheme': sch.stem, 'new_ST': new_ST(sch.stem), 'new_alleles': new_alleles(sch.stem)}
    extra_fastmlst.append(datadict)


In [46]:
df_extra_fastmlst = pd.DataFrame(extra_fastmlst)
df_extra_fastmlst.set_index('Scheme', inplace=True)
df_extra_fastmlst.to_csv('extra_info_fastmlst.csv')

In [29]:
# sum([len(x) for x in data_novel])
print('New ST:', sum([len(x) for x in data_novelST]))
print('New alleles:', sum([len(x) for x in data_novelallele]))


New ST: 1163
New alleles: 3453


In [36]:
#Description assembly-stats
astats = pd.read_csv('assembly-stats.tsv', sep='\t')
astats.groupby('Scheme').describe().to_csv('assembly-stats-description.scv')